# AP2 - Display Data

## Single Skeleton Animation

In [1]:
from Utils import parsXlsxToDf
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib qt
# Load data
df_skeleton = parsXlsxToDf("3_output_depth3d/20230509155025930_3_PRJSAYBY_001_3d_interpolated.xlsx")

# Define pose joints and pose pose
pose_pose = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
             'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
             'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
             'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE',
             'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']
pose_joints = [(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17),
               (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4),
               (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3),
               (11, 12), (27, 29), (13, 15)]

# Extract coordinates from dataframe
coordinates_x = []
coordinates_y = []
coordinates_z = []
coordinates_interpolated = []

for i in range(len(df_skeleton)):
    list_x = []
    list_y = []
    list_z = []
    list_interpolated = []
    for j in range(len(pose_pose)):
        # Hier 0,1,2 für 2D mit Mediapipe Z-Wert, 0,1,4 für 2D mit Kinect Z-Wert oder 5,6,4 für 3D-Koordinaten auswählen, 7,8,9 für Weltkoordinaten
        list_x.append(df_skeleton.iloc[i][pose_pose[j]][7])
        list_y.append(df_skeleton.iloc[i][pose_pose[j]][8])
        list_z.append(df_skeleton.iloc[i][pose_pose[j]][9])
        list_interpolated.append(df_skeleton.iloc[i][pose_pose[j]][10])
    coordinates_x.append(list_x)
    coordinates_y.append(list_y)
    coordinates_z.append(list_z)
    coordinates_interpolated.append(list_interpolated)

# Create figure and axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Set up plot elements
skeleton, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='blue')
interpolated, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='red')
lines = []

for count in pose_joints:
    line = ax.plot([], [], [], '-', color='b', lw='0.5')[0]
    lines.append(line)

ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_xlim(0, 2000)
ax.set_ylim(-1200, 850)
ax.set_zlim(500, 4000)
ax.view_init(elev=135, azim=39)
frame_text = ax.text(0.05, 0.95, 0, "", transform=ax.transAxes)

# Initialize animation
def init():
    skeleton.set_data([], [])
    skeleton.set_3d_properties([])
    for line in lines:
        line.set_data([], [])
        line.set_3d_properties([])
    frame_text.set_text("")
    return skeleton, lines, frame_text

# Update animation
def update(frame):
    zero_indexes = [i for i, (x, y, z) in enumerate(zip(coordinates_x[frame], coordinates_y[frame], coordinates_z[frame])) if x == 0 and y == 0 and z == 0]
    non_zero_x = [coord for i, coord in enumerate(coordinates_x[frame]) if i not in zero_indexes]
    non_zero_y = [coord for i, coord in enumerate(coordinates_y[frame]) if i not in zero_indexes]
    non_zero_z = [coord for i, coord in enumerate(coordinates_z[frame]) if i not in zero_indexes]
    non_interpolated_x = [coord for i, coord in enumerate(non_zero_x) if not coordinates_interpolated[frame][i]]
    non_interpolated_y = [coord for i, coord in enumerate(non_zero_y) if not coordinates_interpolated[frame][i]]
    non_interpolated_z = [coord for i, coord in enumerate(non_zero_z) if not coordinates_interpolated[frame][i]]
    interpolated_x = [coord for i, coord in enumerate(non_zero_x) if coordinates_interpolated[frame][i]]
    interpolated_y = [coord for i, coord in enumerate(non_zero_y) if coordinates_interpolated[frame][i]]
    interpolated_z = [coord for i, coord in enumerate(non_zero_z) if coordinates_interpolated[frame][i]]

    skeleton.set_data(non_interpolated_x, non_interpolated_y)
    skeleton.set_3d_properties(non_interpolated_z)
    interpolated.set_data(interpolated_x, interpolated_y)
    interpolated.set_3d_properties(interpolated_z)

    for lnum, line in enumerate(lines):
        first_index = pose_joints[lnum][0]
        second_index = pose_joints[lnum][1]
        if (first_index in zero_indexes) or (second_index in zero_indexes):
            line.set_data([], [])
            line.set_3d_properties([])
        else:
            first_coord = [coordinates_x[frame][first_index], coordinates_y[frame][first_index], coordinates_z[frame][first_index]]
            second_coord = [coordinates_x[frame][second_index], coordinates_y[frame][second_index], coordinates_z[frame][second_index]]
            line.set_data([first_coord[0], second_coord[0]], [first_coord[1], second_coord[1]])
            line.set_3d_properties([first_coord[2], second_coord[2]])

    frame_text.set_text("Frame: {}".format(frame))
    return skeleton, lines, interpolated, frame_text

# Initialize and show animation
ani = FuncAnimation(fig, update, frames=range(len(coordinates_x)), init_func=init, blit=False)
plt.show()

C:\Users\admin\AppData\Local\Temp\ipykernel_16700\2631065466.py:46: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-o" (-> linestyle='-'). The keyword argument will take precedence.
  skeleton, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='blue')
C:\Users\admin\AppData\Local\Temp\ipykernel_16700\2631065466.py:47: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-o" (-> linestyle='-'). The keyword argument will take precedence.
  interpolated, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='red')


## Two Skeleton Animation

In [2]:
from Utils import parsXlsxToDf
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

%matplotlib qt
import pandas as pd
pose_pose = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
         'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
         'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
         'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE',
         'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']
# Test mit Koordinaten
df_skeleton1 = parsXlsxToDf("3_output_depth3d/20230509155025930_3_PRJSAYBY_001_3d_interpolated.xlsx")
df_skeleton2 = parsXlsxToDf("3_output_depth3d/20230509155047633_3_PRJSAYBY_001_3d_interpolated.xlsx")
i = 0
# Skeleton 1
coordinates_x = []
coordinates_y = []
coordinates_z = []
coordinates_interpolated = []
# Skeleton 2
coordinates_x2 = []
coordinates_y2 = []
coordinates_z2 = []
coordinates_interpolated2 = []
while i < len(df_skeleton1):
    list_x = []
    list_y = []
    list_z = []
    list_interpolated = []
    for j in range(len(pose_pose)):
        # Hier 0,1,2 für 2D mit Mediapipe Z-Wert, 0,1,4 für 2D mit Kinect Z-Wert oder 5,6,4 für 3D-Koordinaten auswählen, 7,8,9 für Weltkoordinaten
        list_x.append(df_skeleton1.iloc[i][pose_pose[j]][7])
        list_y.append(df_skeleton1.iloc[i][pose_pose[j]][8])
        list_z.append(df_skeleton1.iloc[i][pose_pose[j]][9])
        list_interpolated.append(df_skeleton1.iloc[i][pose_pose[j]][10])
    coordinates_x.append(list_x)
    coordinates_y.append(list_y)
    coordinates_z.append(list_z)
    coordinates_interpolated.append(list_interpolated)
    i += 1
i = 0
while i < len(df_skeleton2):

    list_x2 = []
    list_y2 = []
    list_z2 = []
    list_interpolated2 = []
    for j in range(len(pose_pose)):
        # Hier 0,1,2 für 2D mit Mediapipe Z-Wert, 0,1,4 für 2D mit Kinect Z-Wert oder 5,6,4 für 3D-Koordinaten auswählen, 7,8,9 für Weltkoordinaten
        list_x2.append(df_skeleton2.iloc[i][pose_pose[j]][7])
        list_y2.append(df_skeleton2.iloc[i][pose_pose[j]][8])
        list_z2.append(df_skeleton2.iloc[i][pose_pose[j]][9])
        list_interpolated2.append(df_skeleton2.iloc[i][pose_pose[j]][10])
    coordinates_x2.append(list_x2)
    coordinates_y2.append(list_y2)
    coordinates_z2.append(list_z2)
    coordinates_interpolated2.append(list_interpolated2)
    i += 1

pose_pose = ['NOSE', 'LEFT_EYE_INNER', 'LEFT_EYE', 'LEFT_EYE_OUTER', 'RIGHT_EYE_INNER', 'RIGHT_EYE', 'RIGHT_EYE_OUTER',
             'LEFT_EAR', 'RIGHT_EAR', 'MOUTH_LEFT', 'MOUTH_RIGHT', 'LEFT_SHOULDER', 'RIGHT_SHOULDER', 'LEFT_ELBOW',
             'RIGHT_ELBOW', 'LEFT_WRIST', 'RIGHT_WRIST', 'LEFT_PINKY', 'RIGHT_PINKY', 'LEFT_INDEX', 'RIGHT_INDEX',
             'LEFT_THUMB', 'RIGHT_THUMB', 'LEFT_HIP', 'RIGHT_HIP', 'LEFT_KNEE', 'RIGHT_KNEE', 'LEFT_ANKLE',
             'RIGHT_ANKLE', 'LEFT_HEEL', 'RIGHT_HEEL', 'LEFT_FOOT_INDEX', 'RIGHT_FOOT_INDEX']
pose_joints = [(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17),
               (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4),
               (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3),
               (11, 12), (27, 29), (13, 15)]

# Create figure and axis
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})

# MUSS AN AUFLÖSUNG DES VIDEOS ANGEPASST WERDEN
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
#ax.set_xlim(-2500, 0)
#ax.set_ylim(-1500, 0)
ax.set_xlim(0, 2000)
ax.set_ylim(-1200, 850)
ax.set_zlim(500, 4000)

# Set up plot elements
skeleton, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='blue')
interpolated, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='red')
lines = []
for count in pose_joints:
    line = ax.plot([], [], [], '-', color='b', lw='0.5')[0]
    lines.append(line)

skeleton2, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='green')
interpolated2, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='red')
lines2 = []
for count in pose_joints:
    line2 = ax.plot([], [], [], '-', color='g', lw='0.5')[0]
    lines2.append(line2)

ax.view_init(elev=135, azim=39)
frame_text = ax.text(0.05, 0.95, 0, "", transform=ax.transAxes)

# Initialize animation
def init():
    skeleton.set_data([], [])
    skeleton.set_3d_properties([])
    for line in lines:
        line.set_data([], [])
        line.set_3d_properties([])
    skeleton2.set_data([], [])
    skeleton2.set_3d_properties([])
    for line2 in lines2:
        line2.set_data([], [])
        line2.set_3d_properties([])
    frame_text.set_text("")
    return skeleton, lines, frame_text, skeleton2, lines2


# Update animation
# Update animation
def update(frame):
    zero_indexes = [i for i, (x, y, z) in enumerate(zip(coordinates_x[frame], coordinates_y[frame], coordinates_z[frame])) if x == 0 and y == 0 and z == 0]
    non_zero_x = [coord for i, coord in enumerate(coordinates_x[frame]) if i not in zero_indexes]
    non_zero_y = [coord for i, coord in enumerate(coordinates_y[frame]) if i not in zero_indexes]
    non_zero_z = [coord for i, coord in enumerate(coordinates_z[frame]) if i not in zero_indexes]
    non_interpolated_x = [coord for i, coord in enumerate(non_zero_x) if not coordinates_interpolated[frame][i]]
    non_interpolated_y = [coord for i, coord in enumerate(non_zero_y) if not coordinates_interpolated[frame][i]]
    non_interpolated_z = [coord for i, coord in enumerate(non_zero_z) if not coordinates_interpolated[frame][i]]
    interpolated_x = [coord for i, coord in enumerate(non_zero_x) if coordinates_interpolated[frame][i]]
    interpolated_y = [coord for i, coord in enumerate(non_zero_y) if coordinates_interpolated[frame][i]]
    interpolated_z = [coord for i, coord in enumerate(non_zero_z) if coordinates_interpolated[frame][i]]
    skeleton.set_data(non_interpolated_x, non_interpolated_y)
    skeleton.set_3d_properties(non_interpolated_z)
    interpolated.set_data(interpolated_x, interpolated_y)
    interpolated.set_3d_properties(interpolated_z)

    zero_indexes2 = [i for i, (x, y, z) in enumerate(zip(coordinates_x2[frame], coordinates_y2[frame], coordinates_z2[frame])) if x == 0 and y == 0 and z == 0]
    non_zero_x2 = [coord for i, coord in enumerate(coordinates_x2[frame]) if i not in zero_indexes2]
    non_zero_y2 = [coord for i, coord in enumerate(coordinates_y2[frame]) if i not in zero_indexes2]
    non_zero_z2 = [coord for i, coord in enumerate(coordinates_z2[frame]) if i not in zero_indexes2]
    non_interpolated_x2 = [coord for i, coord in enumerate(non_zero_x2) if not coordinates_interpolated2[frame][i]]
    non_interpolated_y2 = [coord for i, coord in enumerate(non_zero_y2) if not coordinates_interpolated2[frame][i]]
    non_interpolated_z2 = [coord for i, coord in enumerate(non_zero_z2) if not coordinates_interpolated2[frame][i]]
    interpolated_x2 = [coord for i, coord in enumerate(non_zero_x2) if coordinates_interpolated2[frame][i]]
    interpolated_y2 = [coord for i, coord in enumerate(non_zero_y2) if coordinates_interpolated2[frame][i]]
    interpolated_z2 = [coord for i, coord in enumerate(non_zero_z2) if coordinates_interpolated2[frame][i]]

    skeleton2.set_data(non_interpolated_x2, non_interpolated_y2)
    skeleton2.set_3d_properties(non_interpolated_z2)
    interpolated2.set_data(interpolated_x2, interpolated_y2)
    interpolated2.set_3d_properties(interpolated_z2)

    for lnum, line in enumerate(lines):
        first_index = pose_joints[lnum][0]
        second_index = pose_joints[lnum][1]
        if (first_index in zero_indexes) or (second_index in zero_indexes):
            line.set_data([], [])
            line.set_3d_properties([])
        else:
            first_coord = [coordinates_x[frame][first_index], coordinates_y[frame][first_index], coordinates_z[frame][first_index]]
            second_coord = [coordinates_x[frame][second_index], coordinates_y[frame][second_index], coordinates_z[frame][second_index]]
            line.set_data([first_coord[0], second_coord[0]], [first_coord[1], second_coord[1]])
            line.set_3d_properties([first_coord[2], second_coord[2]])

    for lnum, line2 in enumerate(lines2):
        first_index2 = pose_joints[lnum][0]
        second_index2 = pose_joints[lnum][1]
        if (first_index2 in zero_indexes2) or (second_index2 in zero_indexes2):
            line2.set_data([], [])
            line2.set_3d_properties([])
        else:
            first_coord2 = [coordinates_x2[frame][first_index2], coordinates_y2[frame][first_index2], coordinates_z2[frame][first_index2]]
            second_coord2 = [coordinates_x2[frame][second_index2], coordinates_y2[frame][second_index2], coordinates_z2[frame][second_index2]]
            line2.set_data([first_coord2[0], second_coord2[0]], [first_coord2[1], second_coord2[1]])
            line2.set_3d_properties([first_coord2[2], second_coord2[2]])
    frame_text.set_text("Frame: {}".format(frame))
    return skeleton, lines, interpolated, frame_text, skeleton2, line2, interpolated2

# blit=True crasht den Plot!!!
ani = FuncAnimation(fig, update, frames=range(len(coordinates_x)), init_func=init, blit=False)
# ani.save("results/test.mp4")
plt.show()

#FuncAnimation.save(ani, "results/ohneKoordinatenachsenTausch.mp4")

C:\Users\admin\AppData\Local\Temp\ipykernel_16700\1656039519.py:85: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-o" (-> linestyle='-'). The keyword argument will take precedence.
  skeleton, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='blue')
C:\Users\admin\AppData\Local\Temp\ipykernel_16700\1656039519.py:86: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-o" (-> linestyle='-'). The keyword argument will take precedence.
  interpolated, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='red')
C:\Users\admin\AppData\Local\Temp\ipykernel_16700\1656039519.py:92: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-o" (-> linestyle='-'). The keyword argument will take precedence.
  skeleton2, = ax.plot([], [], [], '-o', markersize=5, linestyle='None', c='green')
C:\Users\admin\AppData\Local\Temp\ipyk